<h1>Описание проекта</h1>
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Импортируем все нужные библиотеки. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

Открываем таблицу и знакомимся с данными

In [2]:
clients = pd.read_csv('banks_data.csv')
clients.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**children** — количество детей в семье <br>
**days_employed** — общий трудовой стаж в днях <br>
**dob_years** — возраст клиента в годах<br>
**education** — уровень образования клиента<br>
**education_id** — идентификатор уровня образования<br>
**family_status** — семейное положение<br>
**family_status_id** — идентификатор семейного положения<br>
**gender** — пол клиента<br>
**income_type** — тип занятости<br>
**debt** — имел ли задолженность по возврату кредитов<br>
**total_income** — ежемесячный доход<br>
**purpose** — цель получения кредита<br>

Первые комментарии:<br>
    1. Отрицательные значения в **days_employed** — общий трудовой стаж в днях, есть пропуски<br>
    2. Непонятная единица измерения в **total_income** — ежемесячный доход. Больше похоже на ежегодный? Уточнить единицы измерения. Длинный десятичный хвост. Есть пропуски<br>
    3. Нет четкой категоризации в **purpose** — цель получения кредита. Потребуется лемматизация.<br>
    4. В **education** — уровень образования клиента  - нужно привести данные к строчным (маленьким) буквам<br>

Посмотрим на таблицу данных внимательнее

In [3]:
clients.info()
clients.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Комментарии:<br>
1. В **children** есть ошибки. Встречаются значения -1(отсутствующая информация?) и 20 (ошибка ввода?)<br>
2. Вероятнее всего, в **days_employed** нужно взять значения по модулю, чтобы избавиться от отрицаельных значений. Но максимальное значение - 401755 дней = 1100 лет. Что влияет и на среднее значение (около 172 лет). Этого не может быть. <br>
3. Минимальное значение возраста клиента **dob_years** - 0 лет. С этим нужно что-то делать. <br>
4. Есть пропуски в **days_employed** и **total_income**

<h2>План работы по подготовке данных:</h2>
1. Определить и заполнить пропущенные значения. Здесь надо понять причину пропусков. Проверить, в одних ли и тех же строках  отсутствуют данные по доходу и стажу. Тогда можно будет понять, чем заполнять пропуски<br>
2. Заменить вещественный тип данных на целочисленный  <br>
3. Обработать аномалии (Количество детей  =20, огромный стаж)<br>
4. Удалить дубликаты <br>
5. Выделить леммы в значениях столбца с целями получения кредита <br>
6. Категоризировать данные. 


In [4]:
clients[(clients['total_income'].isnull() == True) & (clients['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Получается, что строки с пропусками в **total_income**, совпадают со строками в **days_employed**. Заполним эти значения средним по группе занятости. Очень большие значения в **days_employed**, очевидно. имеют другую единицу измерения - переведем из часов в дни. Отрицательные значения в **days_employed** возьмем по модулю. 

In [5]:
clients['days_employed'] = clients['days_employed'].apply(abs)
clients.loc[clients['days_employed'] >30000, 'days_employed'] = clients.loc[clients['days_employed'] >30000, 'days_employed'] / 24
clients['days_employed'] = clients.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*clients['dob_years']*365))
clients['total_income'] = clients.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Заменим данные в education на нижний регистр

In [6]:
clients['education'] = clients['education'].str.lower()

Зарплату и стаж в днях переведем в целые числа

In [7]:
clients['days_employed'] = clients['days_employed'].astype('int')
clients['total_income'] = clients['total_income'].astype('int')

In [8]:
clients.dtypes

children             int64
days_employed        int32
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int32
purpose             object
dtype: object

Почистим поле с количеством детей. Много ли строк со значением -1? Много ли строк со значением 20?

In [9]:
print("Количество строк с -1", clients[clients['children'] == -1].count()[0])
print("Количество строк с 20", clients[clients['children'] == 20].count()[0])

Количество строк с -1 47
Количество строк с 20 76


-1 = ошибочное значение. Заменим на 0. С 20 - будем считать, что детей 2 или более

In [10]:
clients['children'] = clients['children'].replace(-1, 0)

In [11]:
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,2.152500e+04,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.541092,9.053307e+06,43.293380,0.817236,0.972544,0.080883,1.673954e+05
std,1.379943,4.563662e+07,12.574584,0.548138,1.420324,0.272661,9.790696e+04
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1.024000e+03,33.000000,1.000000,0.000000,0.000000,1.077980e+05
50%,0.000000,2.605000e+03,42.000000,1.000000,0.000000,0.000000,1.519310e+05
75%,1.000000,1.392200e+04,53.000000,1.000000,1.000000,0.000000,2.024170e+05
max,20.000000,4.052299e+08,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь проверим дубликаты

In [12]:
clients.duplicated().sum()

71

In [13]:
clients[clients.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17787,0,299759117,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,299759117,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,310861306,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,310861306,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
13300,0,310861306,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,34681788,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба
9920,0,39306026,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
15991,0,39306026,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
2254,0,41618145,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью


Можно удалять, строки совпадают

In [14]:
clients = clients.drop_duplicates()
clients.duplicated().sum()

0

Сделаем лемматизацию по целям. Получим список уникальных целей

In [15]:
unique_purposes = clients['purpose'].value_counts().index.tolist()
print(unique_purposes)

['свадьба', 'на проведение свадьбы', 'сыграть свадьбу', 'операции с недвижимостью', 'покупка коммерческой недвижимости', 'операции с жильем', 'покупка жилья для сдачи', 'операции с коммерческой недвижимостью', 'покупка жилья', 'жилье', 'покупка жилья для семьи', 'строительство собственной недвижимости', 'недвижимость', 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'строительство недвижимости', 'ремонт жилью', 'покупка жилой недвижимости', 'на покупку своего автомобиля', 'заняться высшим образованием', 'автомобиль', 'сделка с подержанным автомобилем', 'на покупку подержанного автомобиля', 'свой автомобиль', 'автомобили', 'на покупку автомобиля', 'приобретение автомобиля', 'дополнительное образование', 'сделка с автомобилем', 'высшее образование', 'образование', 'получение дополнительного образования', 'получение образования', 'профильное образование', 'получение высшего образования', 'заняться образованием']


проведем лемматизацию с помощью модуля pymystem

In [16]:
m = Mystem()
lemmas = m.lemmatize(' '.join(clients['purpose'].unique()))


In [17]:
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [18]:
Counter(lemmas).most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

Категоризируем цели. Вручную выделим самые частые цели:
1. Недвижимость (сюда же жилье и операции, т.к.имеются в виду операции с недвижимостью)
2. Автомобиль 
3. Образование
4. Свадьба <br>
Создадим новый столбец с типом цели.


In [19]:
purposes_list=['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']
purposes_dict = {v:k for k,v in enumerate(purposes_list)}
purposes_dict['недвижим'] = 1
print(purposes_dict)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


In [20]:
clients['purpose_categ'] = 0
for row in range(len(clients)):
    for purpose in purposes_dict:
        if purpose in clients.iloc[row, 11]:
            clients.iloc[row, 12] = purposes_dict[purpose]
        
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categ
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1


Проверка: остались ли где-то не заполнены категории цели

In [21]:
clients[clients['purpose_categ'] == 0].count().sum()

0

Продолжим категоризацию. Теперь определим категории по доходу, количеству детей, возрасту и стажу. 
По доходу используем категории из описания таблицы (25%, 50, 75 и 100). Для меня так и осталось загадкой, что тут за суммы и в каких единицах, поэтому работаю от среднего значения mean, округлив его

In [22]:
clients['total_income'].describe()

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [23]:
def income_cat(row):
    if row['total_income'] <= 80000:
        return 'низкий'
    elif 80000 < row['total_income'] <= 167000:
        return 'средний'
    elif 167000 < row['total_income'] < 202000:
        return 'высокий'
    else:
        return 'очень высокий'

Вызовем функции, создав новые столбцы

In [24]:
clients['income_cat'] = clients.apply(income_cat, axis=1)
clients.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categ,income_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,очень высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,очень высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,очень высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,средний


Сделала проверку, чтобы посмотреть, равными ли получились категории. Вижу, что нет, поэтому откорректирую функцию

In [25]:
clients.groupby(['income_cat']).size().to_frame('count').reset_index()

,income_cat,count
0,высокий,2711
1,низкий,2276
2,очень высокий,5440
3,средний,11027


In [26]:
def income_cat(row):
    if row['total_income'] <= 100000:
        return 'низкий'
    elif 100000 < row['total_income'] <= 150000:
        return 'средний'
    elif 150000 < row['total_income'] < 202000:
        return 'высокий'
    else:
        return 'очень высокий'

In [27]:
clients['income_cat'] = clients.apply(income_cat, axis=1)
clients.groupby(['income_cat']).size().to_frame('count').reset_index()

,income_cat,count
0,высокий,5461
1,низкий,4463
2,очень высокий,5440
3,средний,6090


Аналогично создадим функции по количеству детей, стажу, возрасту. 

In [28]:
def children_cat(row):
    if row['children'] == 0:
        return 'детей нет '
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные' 
    
def employed_cat(row):
    if row['days_employed'] <= 3650:
        return 'стаж до 10 лет'
    elif 3650 < row['days_employed'] <= 6900:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'
    
def dob_cat(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

In [29]:
clients['employed_cat'] = clients.apply(employed_cat, axis=1)
clients['dob_cat'] = clients.apply(dob_cat, axis=1)
clients['children_cat'] = clients.apply(children_cat, axis=1)
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categ,income_cat,employed_cat,dob_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,очень высокий,стаж более 30 лет,30-45 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,средний,стаж 10-30 лет,30-45 лет,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,средний,стаж 10-30 лет,30-45 лет,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,очень высокий,стаж 10-30 лет,30-45 лет,многодетные
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,высокий,стаж более 30 лет,45-65 лет,детей нет
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,очень высокий,стаж до 10 лет,до 30 лет,детей нет
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,очень высокий,стаж до 10 лет,30-45 лет,детей нет
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3,средний,стаж до 10 лет,45-65 лет,детей нет
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4,низкий,стаж более 30 лет,30-45 лет,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,средний,стаж до 10 лет,30-45 лет,детей нет


<h2> Ответы на вопросы 


<h3>Есть ли зависимость между наличием детей и возвратом кредита в срок?</h3>

In [30]:
report_children = clients.pivot_table(index='children_cat',columns='debt',values='total_income',aggfunc='count')
report_children.columns=['no_debt','debt']
report_children['%']=report_children['debt']/(report_children['debt']+report_children['no_debt'])
report_children

,no_debt,debt,%
children_cat,,,
1-2 ребенка,6222,638,0.093003
детей нет,13074,1064,0.075258
многодетные,417,39,0.085526


Вывод: процент невозврата самый высокий у тех, у кого 1-2 ребенка. Самые надежные заемщики - без детей.

In [31]:
# КОД РЕВЬЮЕРА

clients.groupby('children_cat')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
children_cat,,,
1-2 ребенка,6860,638,9.30%
детей нет,14138,1064,7.53%
многодетные,456,39,8.55%


<h3>Есть ли зависимость между семейным положением и возвратом кредита в срок?</h3>

In [32]:
report_family = clients.pivot_table(index='family_status',columns='debt',values='total_income',aggfunc='count')
report_family.columns=['no_debt','debt']
report_family['%']=report_family['debt']/(report_family['debt']+report_family['no_debt'])
report_family

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2536,274,0.097509
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452


Вывод: лучше всего возвращают кредиты овдовевшие. хуже - те, кто не состоит в официальном браке. 

<h3>Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</h3>

In [33]:
report_income = clients.pivot_table(index='income_cat',columns='debt',values='total_income',aggfunc='count')
report_income.columns=['no_debt','debt']
report_income['%']=report_income['debt']/(report_income['debt']+report_income['no_debt'])
report_income

,no_debt,debt,%
income_cat,,,
высокий,4978,483,0.088445
низкий,4109,354,0.079319
очень высокий,5062,378,0.069485
средний,5564,526,0.086371


Вывод: лучшие клиенты - с очень высоким уровнем дохода. Худшие возвраты у клиентов с высоким и средним доходом.

In [34]:
clients.groupby('income_cat')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
income_cat,,,
высокий,5461,483,8.84%
низкий,4463,354,7.93%
очень высокий,5440,378,6.95%
средний,6090,526,8.64%


<h3>Как разные цели кредита влияют на его возврат в срок?</h3>

In [35]:
report_purpose = clients.pivot_table(index='purpose_categ',columns='debt',values='total_income',aggfunc='count')
report_purpose.columns=['no_debt','debt']
report_purpose['%']=report_purpose['debt']/(report_purpose['debt']+report_purpose['no_debt'])
report_purpose

,no_debt,debt,%
purpose_categ,,,
1,10029,782,0.072334
2,3903,403,0.093590
3,3643,370,0.092200
4,2138,186,0.080034


Вывод: Больше невозвратов у кредитов, взятых на образование и автомобиль. Лучше всего возвращаются кредиты на недвижимость. 

<h2>Общие выводы</h2>

<h2>Вероятнее всего вернет кредит:</h2> Клиент с очень высоким уровнем дохода (6,9% невозврата), без детей (7,5% невозврата), вдовец или разведенный (7,1 и 6,5%) , желающий приобрести недвижимость (7,2% невозврата). 
<h2>Вероятнее всего не вернет кредит:</h2> Клиент с высоким или средним доходом (8,8% и 8,6% невозврата соответственно), не состоящий в браке (9,8%), но имеющий 1-2 детей (9,3%), который собирается купить автомобиль (9,4%) или заняться образованием (9,2% невозврата).